In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

use_device='/gpu:0'


import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import keras

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Input, Concatenate, BatchNormalization
from tensorflow.keras.layers import LSTM

from tensorflow.python.client import device_lib

#import models and layers from tensorflow.keras to avoid errors like "TypeError: Checkpointable._track_checkpointable()"
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

from tensorflow.keras.layers import Dropout, Activation, Flatten
from keras.optimizers import Adam
    
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
    
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#import tensorflow.keras.backend as K
from keras import regularizers
    
import pickle
import pandas as pd

from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
    
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

C:\Users\Alejandro 2\Anaconda3\envs\original_v1_1\envs\original_v1_0\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Alejandro 2\Anaconda3\envs\original_v1_1\envs\original_v1_0\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Alejandro 2\Anaconda3\envs\original_v1_1\envs\original_v1_0\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([

TensorFlow version: 1.13.1


Using TensorFlow backend.
C:\Users\Alejandro 2\Anaconda3\envs\original_v1_1\envs\original_v1_0\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7478279918050535559
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3072013108
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15096198873098384774
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [2]:
# explicitly define paths and functions for loading models, define names, etc
# ========================================================================================================================   
path_data= "xxx"
path_data_2= "xxx"



def load_model_from_path(model_path):
    """
    Load a Keras model from the specified path.
    Parameters:
        model_path (str): The path to the model file.
    Returns:
        model: The loaded Keras model.
    """
    try:
        model = load_model(model_path)
        print("Model loaded successfully.")
        return model
    except Exception as e:
        print("Error loading model:", e)
        


models_names= [
    "LogP_embedding_v1_01.model",
    "D_embedding_v1_01.model",
    "P_embedding_v1_01.model",
    "H_embedding_v1_01.model",
    "pKa_embedding_v1_01.model"]

models_names_act= [
    "LogP_embedding_Acts_v1_01.model",
    "D_embedding_Acts_v1_01.model",
    "P_embedding_Acts_v1_01.model",
    "H_embedding_Acts_v1_01.model",
    "pKa_embedding_Acts_v1_01.model"]

In [3]:
# import data
# ========================================================================================================================  
X1_import=path_data_2 + "SMILES_Tg.txt" #where is the Xdata of the dataset (SMILES)
Y_import=path_data_2 + "Tg.txt" #where is the Ydata of the dataset (property to predict)
   

# Load SMILES strings from file
with open(X1_import, 'r') as f:
    smiles_data_1 = f.readlines()


# Load properties from file
with open(Y_import, 'r') as f:
    X_data = f.readlines()
    
print(smiles_data_1[10])
print(X_data[10])

OC[C@H]1OC(O)[C@H](O)[C@@H](O)[C@H]1O

311.15



In [4]:
# Convert the dataset into activation arrays with the pre-trained models
# ======================================================================================================================== 
# Load the tokenizer
tokenizer_path = path_data_2 + 'tokenizer_all.pkl'
with open(tokenizer_path, 'rb') as f:
    tokenizer_all = pickle.load(f)

# Preprocess SMILES data for SMILES_1
tokenizer_all.fit_on_texts(smiles_data_1)
sequences_1 = tokenizer_all.texts_to_sequences(smiles_data_1)

# Check the maximum index in sequences
max_index_1 = max(max(seq) for seq in sequences_1 if seq)
print(f"Maximum index in sequence 1: {max_index_1}")

# Convert property data to numpy array
X_labels = np.array([float(prop.strip()) for prop in X_data])

# Define vocabulary size for SMILES_1
vocab_size_1 = len(tokenizer_all.word_index) + 1
# Define vocabulary size for SMILES_2
vocab_size_2 = len(tokenizer_all.word_index) + 1
# Define embedding dimension
embedding_dim = 128


max_len=[168, 64, 64, 64, 88] #each model has a different padding


# Load each model, pass the X dataset, and store the activation arrays
acts_dict={}
for i in range(len(models_names_act)):
    model_name=models_names_act[i]
    print(model_name)
    try:
        model_path = path_data_2 + model_name
        model_temp = load_model_from_path(model_path)
        
        # Predict output 1
        max_length_1 = max_len[i]
        padded_sequences_1 = pad_sequences(sequences_1, maxlen=max_length_1, padding='post')
        acts_1 = model_temp.predict(padded_sequences_1)
        print(np.shape(acts_1))
        
        # Store activation arrays for the model
        acts_dict[model_name + "_acts_1"] = acts_1
        
    except Exception as e:
        print("Error loading model:", e)


Maximum index in sequence 1: 31
LogP_embedding_Acts_v1_01.model
Instructions for updating:
Colocations handled automatically by placer.
Model loaded successfully.
(501, 50)
D_embedding_Acts_v1_01.model
Model loaded successfully.
(501, 50)
P_embedding_Acts_v1_01.model
Model loaded successfully.
(501, 50)
H_embedding_Acts_v1_01.model
Model loaded successfully.
(501, 50)
pKa_embedding_Acts_v1_01.model
Model loaded successfully.
(501, 50)


In [7]:
# Extract the data
X11 = acts_dict["LogP_embedding_Acts_v1_01.model_acts_1"]
X12 = acts_dict["D_embedding_Acts_v1_01.model_acts_1"]
X13 = acts_dict["P_embedding_Acts_v1_01.model_acts_1"]
X14 = acts_dict["H_embedding_Acts_v1_01.model_acts_1"]
X15 = acts_dict["pKa_embedding_Acts_v1_01.model_acts_1"]


# Labels
y = X_labels


# Split each set of data into training and test sets
test_size = 0.9
random_state = 42
X11_train, X11_test, y_train, y_test, smiles_train, smiles_test = train_test_split(
    X11, y, smiles_data_1, test_size=test_size, random_state=random_state)

X12_train, X12_test = train_test_split(X12, test_size=test_size, random_state=random_state)
X13_train, X13_test = train_test_split(X13, test_size=test_size, random_state=random_state)
X14_train, X14_test = train_test_split(X14, test_size=test_size, random_state=random_state)
X15_train, X15_test = train_test_split(X15, test_size=test_size, random_state=random_state)


X_train_list = [X11_train, X12_train, X13_train, X14_train, X15_train]

print(np.shape(X11_train))
print(np.shape(X11_test))

(50, 50)
(451, 50)


In [9]:
# Define the test data arrays
X_test_list = [X11_test, X12_test, X13_test, X14_test, X15_test]


# Save each array to a separate .npy file
for idx, array in enumerate(X_test_list):
    np.save(path_data_2 + f'X_test_{idx}.npy', array)    
# Save y_test to a .npy file
np.save(path_data_2 + 'y_test.npy', y_test)
# Save the raw SMILES test data as an npy file
np.save(path_data_2 + 'SMILES_test.npy', np.array(smiles_test))

In [10]:
from keras.callbacks import Callback

   
    
# checkpoint_filepath = path_data_2 + "Weights.npy" #'checkpoint'
monitor_custom='val_mean_absolute_percentage_error' #val_mean_absolute_error   val_mean_absolute_percentage_error
mode_custom='min'



early_stop= tf.keras.callbacks.EarlyStopping(monitor=monitor_custom,  verbose=3,
                                             mode=mode_custom, patience=500, restore_best_weights=True)

callback_list= [early_stop]


def Model_combined(X_train_list, y_train, alpha, NeuronsFC, lr, beta_1, beta_2, epsilon, decay, amsgrad, 
                   ep, b_s, v_s, Drop, shuffle_dataset, callback_list):
    """
    Function to create, compile and train a combined model with multiple inputs. Returns the trained Keras model.
    """
    
    # Define the neural network with 10 input branches (first 5 component 1 and the others component 2)
    size_input = 50
    N_models = 1
    input_layers = [Input(shape=(size_input,)) for _ in range(N_models*1)]
    # Normalize each input layer before concatenating
    normalized_inputs = [BatchNormalization()(input_layer) for input_layer in input_layers]
    
    merged_1 = normalized_inputs[0]
    
    # Define first network
    A = Dense(NeuronsFC)(merged_1)
    A = tf.keras.layers.LeakyReLU(alpha)(A)
    A = Dropout(Drop)(A)
    A = Dense(NeuronsFC)(A)
    A = tf.keras.layers.LeakyReLU(alpha)(A)
    A = Dropout(Drop)(A)
    A = Dense(NeuronsFC)(A)
    A = tf.keras.layers.LeakyReLU(alpha)(A)
    A = Dropout(Drop)(A)

    output_layer = Dense(1, activation='linear')(A)
    
    # Create model
    MODEL = tf.keras.Model(inputs=input_layers, outputs=output_layer, name="MODEL")
    print(MODEL.summary())
    
    # Compile model
    opt = tf.keras.optimizers.Adam(lr, beta_1, beta_2, epsilon, decay, amsgrad)
    MODEL.compile(loss='mean_absolute_percentage_error', optimizer=opt,
                  metrics=['mean_absolute_error', 'mean_absolute_percentage_error'])
    
    # Train the model
    HISTORY = MODEL.fit(X_train_list[0], y_train, epochs=ep, batch_size=b_s, validation_split=v_s,
              shuffle=shuffle_dataset, callbacks=callback_list)
    
    return MODEL, HISTORY

In [15]:
#DATA=====================================================================================================
Mode=0
#NUMBER OF TEST===========================================================================================
number_of_test = 1
#FREEZE_LAYERS============================================================================================
is_trainable_conv1_1=True #
is_trainable_conv1_2=True #
is_trainable_conv1_3=True #
is_trainable_conv1_4=True #
is_trainable_conv1_5=True
is_trainable_HIDDEN_A1_0=True #the CNN input side of the ANN
is_trainable_HIDDEN_A2_0=True #the Hbonds input side of the ANN (1:True, 2:True, 2:False)
is_trainable_HIDDEN_FC_all=True #concatenated FCs
#HYPERPARAMETERS==========================================================================================
lr=[0.01, 0.001, 0.0001]
beta_1=0.99
beta_2=0.999
epsilon=None
decay=0
amsgrad=True
b_s=[1000,10,5]
samples_on_control=0
samples_on_test=200
v_s= 0.3 #(samples_on_test+samples_on_control)/number_of_samples_X   
shuffle_dataset="False"
Drop_prob=0.3
alpha=0.05
reg=0.01
#WORKING PARAMETERS======================================================================================
number_epochs=5000 #
#FULLY CONNECTED=========================================================================================
NeuronsFC=[50] # neurons combined FC
NeuronsFC_A2=5 #neurons in the Hbonds input
NeuronsFC_A1=5 #neurons in the CNN input
#CNN=====================================================================================================
w1=1 # must be the same if the model has been pre-trained
w2=6
w3=10
channels_1=32
channels_2=64
channels_3=128

In [17]:
# Iterate over each combination of learning rate and batch size

counter=0
Results=[]
for i in b_s:
    for j in lr:
        for k in NeuronsFC:
            Name_saving="Tg_mols_embedding_" + str(counter) + str(number_of_test)
            print("model name (to save): " + Name_saving)
            MODEL, HISTORY= Model_combined(X_train_list, y_train, alpha, k, j, beta_1, beta_2, epsilon, decay, amsgrad, 
                       number_epochs, i, v_s, Drop_prob, shuffle_dataset, callback_list)
            #save model
            try:
                MODEL.save(path_data_2 + Name_saving + ".model")
                print("Model saved")
            except:
                print("Model saved?")
                try:
                    MODEL.save(path_data_2 + Name_saving + ".model")
                    print("Model now saved!")
                except Exception as e:
                    print("Error saving model:", e)
            
            # Extract the mean absolute error from the training history
            mean_absolute_error_model = HISTORY.history['mean_absolute_error'][-1]  # Considering the last epoch error
            val_mean_absolute_error_model = HISTORY.history['val_mean_absolute_error'][-1]
            mean_absolute_percentage_error_model = HISTORY.history['mean_absolute_percentage_error'][-1]  # Considering the last epoch error
            val_mean_absolute_percentage_error_model = HISTORY.history['val_mean_absolute_percentage_error'][-1]
            # Append the results to the list
            Results.append([counter, j, i, k, mean_absolute_error_model, val_mean_absolute_error_model,
                           mean_absolute_percentage_error_model, val_mean_absolute_percentage_error_model])            
            counter+=1

model name (to save): Tg_mols_embedding_01
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 50)                0         
_________________________________________________________________
batch_normalization_v1_4 (Ba (None, 50)                200       
_________________________________________________________________
dense_16 (Dense)             (None, 50)                2550      
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 50)                0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 50)                2550      
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (Non

Epoch 25/5000
35/35 [==============================] - 0s 914us/sample - loss: 48.3367 - mean_absolute_error: 119.0830 - mean_absolute_percentage_error: 48.3367 - val_loss: 33.2615 - val_mean_absolute_error: 90.4977 - val_mean_absolute_percentage_error: 33.2615
Epoch 26/5000
35/35 [==============================] - 0s 943us/sample - loss: 53.1298 - mean_absolute_error: 134.8105 - mean_absolute_percentage_error: 53.1298 - val_loss: 32.5798 - val_mean_absolute_error: 89.1539 - val_mean_absolute_percentage_error: 32.5798
Epoch 27/5000
35/35 [==============================] - 0s 857us/sample - loss: 56.1711 - mean_absolute_error: 148.0082 - mean_absolute_percentage_error: 56.1711 - val_loss: 31.8489 - val_mean_absolute_error: 87.5965 - val_mean_absolute_percentage_error: 31.8489
Epoch 28/5000
35/35 [==============================] - 0s 857us/sample - loss: 67.7575 - mean_absolute_error: 178.0367 - mean_absolute_percentage_error: 67.7575 - val_loss: 30.9781 - val_mean_absolute_error: 85.449

Epoch 57/5000
35/35 [==============================] - 0s 543us/sample - loss: 29.9557 - mean_absolute_error: 69.4192 - mean_absolute_percentage_error: 29.9557 - val_loss: 35.5212 - val_mean_absolute_error: 113.2988 - val_mean_absolute_percentage_error: 35.5212
Epoch 58/5000
35/35 [==============================] - 0s 514us/sample - loss: 30.3252 - mean_absolute_error: 68.0996 - mean_absolute_percentage_error: 30.3252 - val_loss: 33.7527 - val_mean_absolute_error: 106.9887 - val_mean_absolute_percentage_error: 33.7527
Epoch 59/5000
35/35 [==============================] - 0s 486us/sample - loss: 31.5901 - mean_absolute_error: 78.9695 - mean_absolute_percentage_error: 31.5901 - val_loss: 32.2815 - val_mean_absolute_error: 101.1831 - val_mean_absolute_percentage_error: 32.2815
Epoch 60/5000
35/35 [==============================] - 0s 543us/sample - loss: 32.8786 - mean_absolute_error: 68.5652 - mean_absolute_percentage_error: 32.8786 - val_loss: 30.7004 - val_mean_absolute_error: 94.9332

Epoch 89/5000
35/35 [==============================] - 0s 400us/sample - loss: 23.4575 - mean_absolute_error: 51.6180 - mean_absolute_percentage_error: 23.4575 - val_loss: 34.8457 - val_mean_absolute_error: 111.4777 - val_mean_absolute_percentage_error: 34.8457
Epoch 90/5000
35/35 [==============================] - 0s 429us/sample - loss: 24.2115 - mean_absolute_error: 55.6206 - mean_absolute_percentage_error: 24.2115 - val_loss: 33.3671 - val_mean_absolute_error: 106.8983 - val_mean_absolute_percentage_error: 33.3671
Epoch 91/5000
35/35 [==============================] - 0s 457us/sample - loss: 28.4212 - mean_absolute_error: 59.7068 - mean_absolute_percentage_error: 28.4212 - val_loss: 31.8972 - val_mean_absolute_error: 102.3387 - val_mean_absolute_percentage_error: 31.8972
Epoch 92/5000
35/35 [==============================] - 0s 400us/sample - loss: 22.8927 - mean_absolute_error: 55.2453 - mean_absolute_percentage_error: 22.8927 - val_loss: 30.6322 - val_mean_absolute_error: 98.3212

Epoch 121/5000
35/35 [==============================] - 0s 743us/sample - loss: 23.6905 - mean_absolute_error: 54.3011 - mean_absolute_percentage_error: 23.6905 - val_loss: 27.6497 - val_mean_absolute_error: 88.7339 - val_mean_absolute_percentage_error: 27.6497
Epoch 122/5000
35/35 [==============================] - 0s 771us/sample - loss: 20.7737 - mean_absolute_error: 46.9717 - mean_absolute_percentage_error: 20.7737 - val_loss: 26.9900 - val_mean_absolute_error: 86.3944 - val_mean_absolute_percentage_error: 26.9900
Epoch 123/5000
35/35 [==============================] - 0s 714us/sample - loss: 25.5436 - mean_absolute_error: 65.5507 - mean_absolute_percentage_error: 25.5436 - val_loss: 26.3594 - val_mean_absolute_error: 84.1484 - val_mean_absolute_percentage_error: 26.3594
Epoch 124/5000
35/35 [==============================] - 0s 1000us/sample - loss: 23.9999 - mean_absolute_error: 54.8567 - mean_absolute_percentage_error: 23.9999 - val_loss: 26.0571 - val_mean_absolute_error: 82.40

...

35/35 [==============================] - 0s 2ms/sample - loss: 38.3747 - mean_absolute_error: 89.7738 - mean_absolute_percentage_error: 38.3747 - val_loss: 20.7093 - val_mean_absolute_error: 65.6673 - val_mean_absolute_percentage_error: 20.7093
Epoch 908/5000
35/35 [==============================] - 0s 2ms/sample - loss: 22.1049 - mean_absolute_error: 49.8497 - mean_absolute_percentage_error: 22.1049 - val_loss: 20.7536 - val_mean_absolute_error: 65.7501 - val_mean_absolute_percentage_error: 20.7536
Epoch 909/5000
35/35 [==============================] - 0s 2ms/sample - loss: 30.9842 - mean_absolute_error: 63.3132 - mean_absolute_percentage_error: 30.9842 - val_loss: 20.8224 - val_mean_absolute_error: 65.9675 - val_mean_absolute_percentage_error: 20.8224
Epoch 910/5000
35/35 [==============================] - 0s 2ms/sample - loss: 35.0634 - mean_absolute_error: 74.3372 - mean_absolute_percentage_error: 35.0634 - val_loss: 20.8356 - val_mean_absolute_error: 66.0441 - val_mean_absolute_p

35/35 [==============================] - 0s 2ms/sample - loss: 30.0121 - mean_absolute_error: 69.3078 - mean_absolute_percentage_error: 30.0121 - val_loss: 21.6489 - val_mean_absolute_error: 67.9423 - val_mean_absolute_percentage_error: 21.6489
Epoch 970/5000
35/35 [==============================] - 0s 2ms/sample - loss: 33.5889 - mean_absolute_error: 74.0131 - mean_absolute_percentage_error: 33.5889 - val_loss: 21.6462 - val_mean_absolute_error: 68.0146 - val_mean_absolute_percentage_error: 21.6462
Epoch 971/5000
35/35 [==============================] - 0s 2ms/sample - loss: 29.4487 - mean_absolute_error: 62.7971 - mean_absolute_percentage_error: 29.4487 - val_loss: 21.6802 - val_mean_absolute_error: 68.1050 - val_mean_absolute_percentage_error: 21.6802
Epoch 972/5000
35/35 [==============================] - 0s 3ms/sample - loss: 30.1605 - mean_absolute_error: 67.3582 - mean_absolute_percentage_error: 30.1605 - val_loss: 21.6738 - val_mean_absolute_error: 68.0991 - val_mean_absolute_p

35/35 [==============================] - 0s 2ms/sample - loss: 26.7956 - mean_absolute_error: 58.7092 - mean_absolute_percentage_error: 26.7956 - val_loss: 21.5445 - val_mean_absolute_error: 68.3919 - val_mean_absolute_percentage_error: 21.5445
Epoch 1032/5000
35/35 [==============================] - 0s 3ms/sample - loss: 36.0459 - mean_absolute_error: 83.1332 - mean_absolute_percentage_error: 36.0459 - val_loss: 21.4938 - val_mean_absolute_error: 68.2953 - val_mean_absolute_percentage_error: 21.4938
Epoch 1033/5000
35/35 [==============================] - 0s 3ms/sample - loss: 38.5452 - mean_absolute_error: 76.8982 - mean_absolute_percentage_error: 38.5452 - val_loss: 21.4076 - val_mean_absolute_error: 68.1207 - val_mean_absolute_percentage_error: 21.4076
Epoch 1034/5000
35/35 [==============================] - 0s 3ms/sample - loss: 35.0020 - mean_absolute_error: 77.1668 - mean_absolute_percentage_error: 35.0020 - val_loss: 21.3898 - val_mean_absolute_error: 68.1191 - val_mean_absolut

35/35 [==============================] - 0s 3ms/sample - loss: 32.1957 - mean_absolute_error: 67.7167 - mean_absolute_percentage_error: 32.1957 - val_loss: 21.7149 - val_mean_absolute_error: 69.2159 - val_mean_absolute_percentage_error: 21.7149
Epoch 1094/5000
35/35 [==============================] - 0s 3ms/sample - loss: 32.1410 - mean_absolute_error: 73.9842 - mean_absolute_percentage_error: 32.1410 - val_loss: 21.7152 - val_mean_absolute_error: 69.2530 - val_mean_absolute_percentage_error: 21.7152
Epoch 1095/5000
35/35 [==============================] - 0s 3ms/sample - loss: 37.6518 - mean_absolute_error: 82.6675 - mean_absolute_percentage_error: 37.6518 - val_loss: 21.7744 - val_mean_absolute_error: 69.3602 - val_mean_absolute_percentage_error: 21.7744
Epoch 1096/5000
35/35 [==============================] - 0s 2ms/sample - loss: 31.1665 - mean_absolute_error: 69.5808 - mean_absolute_percentage_error: 31.1665 - val_loss: 21.7970 - val_mean_absolute_error: 69.3930 - val_mean_absolut

Epoch 01123: early stopping
Model saved?
Error saving model: Unable to create link (name already exists)


In [18]:
#save as excel
#===============

# Define column names for the Excel file
columns = ['Counter', 'lr', 'bs', "Number of neurons", 'MAE', 'val MAE', 'MAPE', 'val MAPE']
# Convert Results array to a DataFrame
Results_df = pd.DataFrame(Results, columns=columns)
writer = pd.ExcelWriter(path_data_2 + "Results_file_" + str(number_of_test) + ".xlsx", engine='xlsxwriter')
Results_df.to_excel(writer, sheet_name='Results_file_', index=False)
writer.save()

In [19]:
Results_df.head(20)

,Counter,lr,bs,Number of neurons,MAE,val MAE,MAPE,val MAPE
0,0,0.0100,1000,50,37.580593,39.543850,15.676857,14.899499
1,1,0.0010,1000,50,39.852722,37.235359,16.627180,12.848083
2,2,0.0001,1000,50,59.931583,47.595089,23.504602,15.435128
3,3,0.0100,10,50,59.247784,44.806446,23.933012,16.390066
4,4,0.0010,10,50,41.293549,44.942039,18.047470,15.565166
5,5,0.0001,10,50,86.598518,48.806206,40.337151,16.459976
6,6,0.0100,5,50,54.175449,75.392960,21.569080,25.948612
7,7,0.0010,5,50,37.048031,66.513039,16.482693,21.733244
8,8,0.0001,5,50,88.602875,70.167191,43.308971,22.226652
